This notebook subsets the full Yelp dataset, so that we can easily work with it to develop algorithms first. Subsequent analyses is then done on the full dataset. 

# Load Data

In [3]:
import pandas as pd
import json
import progressbar

## Load Reviews
First, load the reviews. We can use chunksize to limit the number of reviews selected.

In [1]:
num_reviews = 300000

In [4]:
df = pd.read_json('data/review.json', lines=True, orient='columns', chunksize=num_reviews)
for chunk in df:
    review = chunk
    break
review.set_index('review_id', inplace=True)
review.head()

,business_id,cool,date,funny,stars,text,useful,user_id
review_id,,,,,,,,
Q1sbwvVQXV2734tPgoKj4Q,ujmEBvifdJM6h6RLv4wQIg,0,2013-05-07 04:34:36,1,1,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA
GJXCdrto3ASJOqKeVWPi6Q,NZnhc2sEQy3RmzKTZnqtwQ,0,2017-01-14 21:30:33,0,5,I *adore* Travis at the Hard Rock's new Kelly ...,0,yXQM5uF2jS6es16SJzNHfg
2TzJjDVDEuAW6MR5Vuc1ug,WTqjgwHlXbSFevF32_DJVw,0,2016-11-09 20:09:03,0,5,I have to say that this office really has it t...,3,n6-Gk65cPZL6Uz8qRm3NYw
yi0R0Ugj_xUx_Nek0-_Qig,ikCg8xy5JIg_NGPx-MSIDA,0,2018-01-09 20:56:38,0,5,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg
11a8sVPMUFtaC7_ABRkmtw,b1b1eb3uo-w561D0ZfCEiQ,0,2018-01-30 23:07:38,0,1,Today was my second out of three sessions I ha...,7,ssoyf2_x0EQMed6fgHeMyQ


## Load Supplementary Data
Next, load the corresponding tables. In order to save memory, we only load the data that is referenced in the reviews table.

In [5]:
business_ids = review.business_id.unique()
user_ids = review.user_id.unique()

In [6]:
def load_data(filename, filters, stop_when_done):
    bar = progressbar.ProgressBar(widgets=[progressbar.AnimatedMarker(), " ", progressbar.Counter(), " ", progressbar.BouncingBar(), " ", progressbar.Timer()])
    i = 0
    df_dict = {}
    with open("data/"+filename+".json", encoding='utf-8') as f:
        for line in f:
            obj = json.loads(line)
            add = True
            for col_to_filter, filter_items in filters:
                if (obj[col_to_filter] not in filter_items):
                    add = False
                    break
            if add:
                df_dict[i] = obj
                i+=1
                if stop_when_done and len(df_dict) == len(filter_items):
                    break
            bar.update(len(df_dict))
    bar.finish()
    return pd.DataFrame.from_dict(df_dict, 'index')

In [7]:
business = load_data('business', [('business_id', business_ids)], True)
business.set_index('business_id', inplace=True)
business.drop(['postal_code', 'is_open', 'attributes', 'categories', 'hours'], axis=1, inplace=True)
business.head()

| 18165 |                 #                             | Elapsed Time: 0:00:20


,name,address,city,state,latitude,longitude,stars,review_count
business_id,,,,,,,,
1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,33.522143,-112.018481,3.0,5
QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,43.605499,-79.652289,2.5,128
gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,35.092564,-80.859132,4.0,170
HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,35.190012,-80.887223,4.0,4
5JucpCfHZltJh5r1JabjDg,Edgeworxx Studio,20 Douglas Woods Drive Southeast,Calgary,AB,50.943646,-114.001828,3.5,7


In [8]:
user = load_data('user', [('user_id', user_ids)], True)
user.set_index('user_id', inplace=True)
user.head()

| 202406 |                                   #          | Elapsed Time: 0:53:45


,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
user_id,,,,,,,,,,,,,,,,,,,,,
l6BmjZMeQD3rDxWUbiAiow,Rashmi,95,2013-10-08 23:11:33,84,17,25,"2015,2016,2017","c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",5,4.03,...,0,0,0,0,1,1,1,1,2,0
4XChL029mKr5hydo79Ljxg,Jenna,33,2013-02-21 22:29:06,48,22,16,,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",4,3.63,...,0,0,0,0,0,0,1,1,0,0
bc8C_eETBWL0olvFSJJd0w,David,16,2013-10-04 00:16:10,28,8,10,,"4N-HU_T32hLENLntsNKNBg, pSY2vwWLgWfGVAAiKQzMng...",0,3.71,...,0,0,0,0,1,0,0,0,0,0
dD0gZpBctWGdWo9WlGuhlA,Angela,17,2014-05-22 15:57:30,30,4,14,,"RZ6wS38wnlXyj-OOdTzBxA, l5jxZh1KsgI8rMunm-GN6A...",5,4.85,...,0,0,0,0,0,2,0,0,1,0
MM4RJAeH6yuaN8oZDSt0RA,Nancy,361,2013-10-23 07:02:50,1114,279,665,"2015,2016,2017,2018","mbwrZ-RS76V1HoJ0bF_Geg, g64lOV39xSLRZO0aQQ6DeQ...",39,4.08,...,1,0,0,1,16,57,80,80,25,5


In [9]:
checkin = load_data('checkin', [('business_id', business_ids)], False)
checkin.set_index('business_id', inplace=True)
checkin.head()

| 15504 |            #                                  | Elapsed Time: 0:03:09


,date
business_id,
--Gc998IMjLn8yr-HTzGUg,2014-07-01 01:20:47
--I7YYLada0tSLkORTHb5Q,"2014-11-07 00:51:45, 2014-11-10 23:51:38, 2014..."
--U98MNlDym2cLn36BBPgQ,"2011-10-05 22:50:41, 2012-04-11 00:06:36, 2012..."
--wIGbLEhlpl_UeAIyDmZQ,2015-06-06 20:01:06
-000aQFeK6tqVLndf7xORg,2018-10-17 21:16:27


## Export Data

We export the data for easier loading for future sessions. 

In [10]:
review.to_parquet("data/"+str(num_reviews)+"_review.parquet")
business.to_parquet("data/"+str(num_reviews)+"_business.parquet")
user.to_parquet("data/"+str(num_reviews)+"_user.parquet")
checkin.to_parquet("data/"+str(num_reviews)+"_checkin.parquet")

In [ ]:
from flask import Flask, send_from_directory, jsonify
import pandas as pd
import numpy as np
import json
import math
import spacy
from tqdm import tqdm
tqdm().pandas()
from wordcloud import WordCloud